## Set system path

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/DLC-Jupyter-Notebooks
sys.path.append(str(project_root))


## Connect to database

In [ ]:
import os
import psycopg2
import pandas as pd
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.49"
conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password="1234", host=host, port="5432")
cursor = conn.cursor()


## Plot a few trajectories side by side

In [ ]:
query = """
SELECT id FROM dlc_table WHERE task='ToyOnly' AND genotype = 'white' AND dose_mult=10
ORDER BY id
"""

df = pd.read_sql_query(query, conn)
id_list = df['id'].tolist()
print(id_list)

from Python_scripts.Data_analysis.plot_single_trajectory import plot_single_trajectory

import matplotlib.pyplot as plt

for id in id_list:
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    plot_single_trajectory(conn, id, bodypart='Midback',
                           normalize=False, interpolate=False, likelihood_threshold=0,
                           label=None, color=None, style='scatter',
                           max_points=None, color_by_time=True, ax=axs[0]
                           )

    plot_single_trajectory(conn, id, bodypart='Midback',
                       normalize=True, interpolate=True, likelihood_threshold=0.5,
                       label=None, color=None, style='line',
                       max_points=None, color_by_time=False, ax=axs[1]
                       )
    plt.tight_layout()
    plt.show()


## Compare DLC vs Optogen

In [ ]:
import importlib
import Python_scripts.Data_analysis.plot_single_trajectory
import Python_scripts.Data_analysis.plot_single_trajectory_optogen

importlib.reload(Python_scripts.Data_analysis.plot_single_trajectory)
importlib.reload(Python_scripts.Data_analysis.plot_single_trajectory_optogen)

from Python_scripts.Data_analysis.plot_single_trajectory import plot_single_trajectory
from Python_scripts.Data_analysis.plot_single_trajectory_optogen import plot_single_trajectory_optogen

query = "SELECT id FROM dlc_table WHERE video_name LIKE '%FoodLight_9_10_24_Indigo_Y%'"

df = pd.read_sql_query(query, conn)
id = df['id'][0]
print(id)
print(type(id))
plot_single_trajectory(conn, id, max_points=500)
plot_single_trajectory_optogen(conn, id, max_points=500)


In [ ]:
import os
import shutil
from pathlib import Path

# Source and destination directories
src_dir = Path("/Users/atanugiri/Downloads/data/FoodLightWhite/DlcDataPytorch")
dst_dir = Path("/Users/atanugiri/Downloads/data/FoodLightWhite/DlcDataPytorchFiltered")

# Create destination directory if it doesn't exist
dst_dir.mkdir(parents=True, exist_ok=True)

# Process all .csv files
for csv_file in src_dir.glob("*.csv"):
    base = csv_file.stem  # filename without .csv extension
    new_name = f"{base}_filtered.csv"
    dst_path = dst_dir / new_name

    shutil.copy(csv_file, dst_path)
    print(f"Copied and renamed: {csv_file.name} -> {new_name}")
